# Inference

This notebook is for generating Kaggle submission file. It currently incorporates two neural networks that perform segmentation and then classification.

## Setup

In [1]:
from matplotlib import pyplot as plt
%load_ext autoreload
%autoreload 2

In [58]:
import pandas as pd
import albumentations as A
import numpy as np

from scripts.models import CoinLocalizer, HierarchicalClassifier
from scripts.training import get_best_available_device, load_params
from scripts.utils import SegmentationDataset, split_data, ClassificationDataset, \
    generate_hough, get_cropped_image, get_segmentation, get_class
from scripts.config import example_row, row_template, ID_TO_LABEL

from PIL import Image
from collections import Counter
from torch.utils.data import DataLoader

## Dataset

Specify the test directory, generate Imagenet specific transforms, initialize the dataset and put it into dataloader.

In [54]:
test_directory = "data/test"
test_image_paths, _, _, _ = split_data(test_directory, 0.0, 'inference')

In [55]:
seg_tf = A.Compose([
    A.Resize(width=600, height=400, always_apply=True),
    A.PadIfNeeded(min_height=416, min_width=608, always_apply=True),
])

cls_tf = A.Compose([
    A.Resize(width=224, height=224, always_apply=True),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), always_apply=True)  # imagenet specific
])

In [56]:
test_ds = SegmentationDataset(
    image_paths=test_image_paths,
    transform=seg_tf,
)

test_loader = DataLoader(test_ds, batch_size=1)

## Models

Initialize segmentation and classification models and load pre-trained weights.

In [57]:
device = get_best_available_device()

seg_model = CoinLocalizer()
seg_model = load_params(seg_model, 'segmentation_.pt')

cls_model = HierarchicalClassifier()

cls_model = cls_model.to(device)
model = seg_model.to(device)

## Inference

Iterate over images, extract coins and predict their class.

In [49]:
# generate empty df for predictions
df = pd.DataFrame(columns=example_row)

In [50]:
id_to_label = np.vectorize(lambda x: ID_TO_LABEL.get(x, "Unknown"))

for i, (image, _, filename) in enumerate(test_loader):

    image = image.to(device)

    predicted = get_segmentation(model, image)

    circles, hough_img = generate_hough(predicted, image)
    
    original_img = Image.open(f"data/test/{filename[0]}")
    original_img = np.array(original_img)
    
    # segmentation was done on smaller images -> reset the coordinates for original images
    x_ratio = 6000 / image.shape[3]
    y_ratio = 4000 / image.shape[2]

    labels = []

    for j, (x, y, r) in enumerate(circles):
        
        cropped_image = get_cropped_image(original_img, x, y, r, x_ratio, y_ratio)
        plt.imshow()
        
        # initiate the dataloader
        coin_loader = DataLoader(ClassificationDataset(cropped_image, transform=cls_tf))
        coin_iterator = iter(coin_loader)
        coin, _, radius = next(coin_iterator)
        
        # predict the class and add it to labels
        coin_label = get_class(cls_model, coin, radius, id_to_label)
        labels.append(coin_label)
    
    # count prediction labels and update template dictionary
    row = row_template.copy()
    new_count = Counter(labels)
    row.update(new_count)
    
    # add row to the df
    new_row_df = pd.DataFrame([row], index=[filename[0].split('.')[0]])
    df = pd.concat([df, new_row_df])

In [51]:
# add index name as necessary for Kaggle
df.index.name='id'
df.head()

,5CHF,2CHF,1CHF,0.5CHF,0.2CHF,0.1CHF,0.05CHF,2EUR,1EUR,0.5EUR,0.2EUR,0.1EUR,0.05EUR,0.02EUR,0.01EUR,OOD
id,,,,,,,,,,,,,,,,
L0000000,0,2,0,0,0,1,0,0,0,0,0,0,0,0,0,0
L0000001,0,4,0,0,2,0,0,1,0,1,2,0,0,0,0,0
L0000002,3,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
L0000003,0,0,1,2,0,0,1,0,0,0,0,1,1,1,0,1
L0000004,0,1,0,1,0,2,0,2,0,0,0,0,0,0,0,0


In [52]:
df.to_csv('test_submission.csv')